In [ ]:
from datetime import date
from pyspark.sql.types import *
from pyspark.sql.functions import lit
import shutil
import os
import subprocess

In [ ]:
# this has been added for scenarios where you might
# wish to alter some of the churn label prediction
# logic but do not wish to rerun the whole notebook
skip_reload = False

# please use a personalized database name here if you wish to avoid interfering with other users who might be running this accelerator in the same workspace
database_name = 'kkbox_churn'
data_dir = f"{os.getenv('HOME')}/databricks/kkbox_churn"

In [ ]:
spark.stop()  # Properly stop Spark
del spark     # Delete the variable

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("ChurnCluster") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
    .config("spark.executor.memory", "56g") \
    .config("spark.driver.memory", "56g") \
    .getOrCreate()

os.environ["SPARK_APP_NAME"] = spark.conf.get("spark.app.name")
os.environ["SPARK_MASTER"] = spark.conf.get("spark.master")

print("Spark Version:", spark.version)

In [ ]:
if skip_reload:
  # create database to house SQL tables
  _ = spark.sql(f'CREATE DATABASE IF NOT EXISTS {database_name}')
  _ = spark.sql(f'USE {database_name}')
else:
  # delete the old database if needed
  _ = spark.sql(f'DROP DATABASE IF EXISTS {database_name} CASCADE')
  _ = spark.sql(f'CREATE DATABASE {database_name}')
  _ = spark.sql(f'USE {database_name}')

  # drop any old delta lake files that might have been created
  folder_path = f'{data_dir}/silver/members'
  if os.path.exists(folder_path):
      shutil.rmtree(folder_path)
    
  # members dataset schema
  member_schema = StructType([
    StructField('msno', StringType()),
    StructField('city', IntegerType()),
    StructField('bd', IntegerType()),
    StructField('gender', StringType()),
    StructField('registered_via', IntegerType()),
    StructField('registration_init_time', DateType())
    ])

  # read data from csv
  members = (
    spark
      .read
      .csv(
        f'{data_dir}/members/members_v3.csv',
        schema=member_schema,
        header=True,
        dateFormat='yyyyMMdd'
        )
      )

  # persist in delta lake format
  (
    members
      .write
      .format('delta')
      .mode('overwrite')
      .save(f'{data_dir}/silver/members')
    )

    # create table object to make delta lake queryable
  _ = spark.sql('''
      CREATE TABLE members 
      USING DELTA 
      LOCATION '/home/dinindu/databricks/kkbox_churn/silver/members'
      ''')


In [ ]:
# print(members.show())
result = spark.sql("SELECT * FROM kkbox_churn.members LIMIT 10")
result.show()

In [ ]:
if not skip_reload:

  # drop any old delta lake files that might have been created
  folder_path = f'{data_dir}/silver/transactions'
  if os.path.exists(folder_path):
      shutil.rmtree(folder_path)

  # transaction dataset schema
  transaction_schema = StructType([
    StructField('msno', StringType()),
    StructField('payment_method_id', IntegerType()),
    StructField('payment_plan_days', IntegerType()),
    StructField('plan_list_price', IntegerType()),
    StructField('actual_amount_paid', IntegerType()),
    StructField('is_auto_renew', IntegerType()),
    StructField('transaction_date', DateType()),
    StructField('membership_expire_date', DateType()),
    StructField('is_cancel', IntegerType())  
    ])

  # read data from csv
  transactions = (
    spark
      .read
      .csv(
        f'{data_dir}/transactions/transactions.csv',
        schema=transaction_schema,
        header=True,
        dateFormat='yyyyMMdd'
        )
      )

  # persist in delta lake format
  ( transactions
      .write
      .format('delta')
      .partitionBy('transaction_date')
      .mode('overwrite')
      .save(f'{data_dir}/silver/transactions')
    )

    # create table object to make delta lake queryable
  _ = spark.sql('''
      CREATE TABLE transactions
      USING DELTA 
      LOCATION '/home/dinindu/databricks/kkbox_churn/silver/transactions'
      ''')

In [ ]:
# print(transactions.show())
result = spark.sql("SELECT * FROM kkbox_churn.transactions LIMIT 10")
result.show()

In [ ]:
if not skip_reload:
  # drop any old delta lake files that might have been created
  folder_path = f'{data_dir}/silver/user_logs'
  if os.path.exists(folder_path):
      shutil.rmtree(folder_path)

  # transaction dataset schema
  user_logs_schema = StructType([ 
    StructField('msno', StringType()),
    StructField('date', DateType()),
    StructField('num_25', IntegerType()),
    StructField('num_50', IntegerType()),
    StructField('num_75', IntegerType()),
    StructField('num_985', IntegerType()),
    StructField('num_100', IntegerType()),
    StructField('num_uniq', IntegerType()),
    StructField('total_secs', FloatType())  
    ])

  # read data from csv
  user_logs = (
    spark
      .read
      .csv(
        f'{data_dir}/user_logs/user_logs.csv',
        schema=user_logs_schema,
        header=True,
        dateFormat='yyyyMMdd'
        )
      )

  # persist in delta lake format
  ( user_logs
      .write
      .format('delta')
      .partitionBy('date')
      .mode('overwrite')
      .save(f'{data_dir}/silver/user_logs')
    )

  # create table object to make delta lake queryable
  _ = spark.sql('''
    CREATE TABLE IF NOT EXISTS user_logs
    USING DELTA 
    LOCATION '/home/dinindu/databricks/kkbox_churn/silver/user_logs'
    ''')

In [ ]:
# Delete training labels if exists before create
_ = spark.sql('DROP TABLE IF EXISTS train')

In [ ]:
%%sh -e

# Create a spark cluster. But not required.
docker network create spark-net

docker run -d --rm --network spark-net --name spark-master \
    -p 8080:8080 -p 7077:7077 -p 4040:4040 \
    bitnami/spark spark-class org.apache.spark.deploy.master.Master

docker run -d --rm --network spark-net --name spark-worker \
    --env SPARK_MODE=worker \
    --env SPARK_MASTER_URL=spark://spark-master:7077 \
    bitnami/spark spark-class org.apache.spark.deploy.worker.Worker spark://spark-master:7077


In [ ]:
# Generate training labels
%%sh -e

kkbox_churn_dir="/home/dinindu/databricks/kkbox_churn"
sudo chmod 777 $kkbox_churn_dir
sudo rm -rf $kkbox_churn_dir/silver/train

docker run --rm --network host  \
    -v "$kkbox_churn_dir:/opt/spark/work/kkbox_churn" \
    -v "$PWD:/opt/bitnami/spark/work" \
    bitnami/spark:3.4.1 spark-shell --master local[*] \
    --executor-memory 48G \
    --driver-memory 16G \
    --packages io.delta:delta-core_2.12:2.4.0 \
    --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension \
    --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog \
    -i /opt/bitnami/spark/work/scripts/generate_training_labels.scala

sudo chown -R dinindu:dinindu $kkbox_churn_dir/silver/train
sudo chmod -R 777 $kkbox_churn_dir/silver/train

In [ ]:
# Access the training labels
_ = spark.sql('''
CREATE TABLE IF NOT EXISTS train
USING DELTA
LOCATION '/home/dinindu/databricks/kkbox_churn/silver/train/'
''')

_ = spark.sql('SELECT * FROM train').show(10)

In [ ]:
# Delete testing labels if exists before create
_ = spark.sql('DROP TABLE IF EXISTS test')

In [ ]:
# Generate testing labels
%%sh -e

kkbox_churn_dir="/home/dinindu/databricks/kkbox_churn"
sudo chmod 777 $kkbox_churn_dir
sudo rm -rf $kkbox_churn_dir/silver/test

docker run --rm --network host  \
    -v "$kkbox_churn_dir:/opt/spark/work/kkbox_churn" \
    -v "$PWD:/opt/bitnami/spark/work" \
    bitnami/spark:3.4.1 spark-shell --master local[*] \
    --executor-memory 48G \
    --driver-memory 16G \
    --packages io.delta:delta-core_2.12:2.4.0 \
    --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension \
    --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog \
    -i /opt/bitnami/spark/work/scripts/generate_testing_labels.scala

sudo chown -R dinindu:dinindu $kkbox_churn_dir/silver/test
sudo chmod -R 777 $kkbox_churn_dir/silver/test

In [ ]:
# Access the testing labels
_ = spark.sql('''
CREATE TABLE IF NOT EXISTS test
USING DELTA
LOCATION '/home/dinindu/databricks/kkbox_churn/silver/test/'
              ''')

_ = spark.sql('SELECT * FROM test').show(10)

Step 3: Cleanse & Enhance Transaction Logs

In the churn script provided by KKBox (and used in the last step), time between transaction events is used in order to determine churn status. In situations where multiple transactions are recorded on a given date, complex logic is used to determine which transaction represents the final state of the account on that date. This logic states that when we have multiple transactions for a given subscriber on a given date, we should:

1. Concatenate the plan_list_price, payment_plan_days, and payment_method_id values and consider the "bigger" of these values as preceding the others
2. If the concatenated value (defined in the last step) is the same across records for this date, cancellations, i.e. records where is_cancel=1, should follow other transactions
3. If there are multiple cancellations in this sequence, the record with the earliest expiration date is the last record for this transaction date
4. If there are no cancellations but multiple non-cancellations in this sequence, the non-cancellation record with the latest expiration date is the last record on the transaction date
Rewriting this logic in SQL allows us to generate a cleansed version of the transaction log with the final record for each date:

In [ ]:
# %pip install ipython-sql
%reload_ext sql
%sql sqlite:////home/dinindu/Projects/dini-ai-ml/kaggle/churn/spark-warehouse/kkbox_churn.db


In [8]:
_ = spark.sql('''
    DROP TABLE IF EXISTS transactions_clean;

    CREATE TABLE transactions_clean
    USING DELTA
    AS
    WITH 
        transaction_sequenced (
        SELECT
            msno,
            transaction_date,
            plan_list_price,
            payment_plan_days,
            payment_method_id,
            is_cancel,
            membership_expire_date,
            RANK() OVER (PARTITION BY msno, transaction_date ORDER BY plan_sort DESC, is_cancel) as sort_id  -- calc rank on price, days & method sort followed by cancel sort
        FROM (
            SELECT
            msno,
            transaction_date,
            plan_list_price,
            payment_plan_days,
            payment_method_id,
            CONCAT(CAST(plan_list_price as string), CAST(payment_plan_days as string), CAST(payment_method_id as string)) as plan_sort,
            is_cancel,
            membership_expire_date
            FROM transactions
            )
        )
    SELECT
        p.msno,
        p.transaction_date,
        p.plan_list_price,
        p.actual_amount_paid,
        p.plan_list_price - p.actual_amount_paid as discount,
        p.payment_plan_days,
        p.payment_method_id,
        p.is_cancel,
        p.is_auto_renew,
        p.membership_expire_date
    FROM transactions p
    INNER JOIN (
        SELECT
        x.msno,
        x.transaction_date,
        x.plan_list_price,
        x.payment_plan_days,
        x.payment_method_id,
        x.is_cancel,
        CASE   -- if is_cancel is 0 in last record then go with max membership date identified, otherwise go with lowest membership date
            WHEN x.is_cancel=0 THEN MAX(x.membership_expire_date)
            ELSE MIN(x.membership_expire_date)
            END as membership_expire_date
        FROM (
        SELECT
            a.msno,
            a.transaction_date,
            a.plan_list_price,
            a.payment_plan_days,
            a.payment_method_id,
            a.is_cancel,
            a.membership_expire_date
        FROM transaction_sequenced a
        INNER JOIN (
            SELECT msno, transaction_date, MAX(sort_id) as max_sort_id -- find last entries on a given date
            FROM transaction_sequenced 
            GROUP BY msno, transaction_date
            ) b
            ON a.msno=b.msno AND a.transaction_date=b.transaction_date AND a.sort_id=b.max_sort_id
            ) x
        GROUP BY 
        x.msno, 
        x.transaction_date, 
        x.plan_list_price,
        x.payment_plan_days,
        x.payment_method_id,
        x.is_cancel
    ) q
    ON 
        p.msno=q.msno AND 
        p.transaction_date=q.transaction_date AND 
        p.plan_list_price=q.plan_list_price AND 
        p.payment_plan_days=q.payment_plan_days AND 
        p.payment_method_id=q.payment_method_id AND 
        p.is_cancel=q.is_cancel AND 
        p.membership_expire_date=q.membership_expire_date;
        
    ''')

ParseException: 
extraneous input 'CREATE' expecting {<EOF>, ';'}(line 4, pos 4)

== SQL ==

    DROP TABLE IF EXISTS transactions_clean;

    CREATE TABLE transactions_clean
----^^^
    USING DELTA
    AS
    WITH 
        transaction_sequenced (
        SELECT
            msno,
            transaction_date,
            plan_list_price,
            payment_plan_days,
            payment_method_id,
            is_cancel,
            membership_expire_date,
            RANK() OVER (PARTITION BY msno, transaction_date ORDER BY plan_sort DESC, is_cancel) as sort_id  -- calc rank on price, days & method sort followed by cancel sort
        FROM (
            SELECT
            msno,
            transaction_date,
            plan_list_price,
            payment_plan_days,
            payment_method_id,
            CONCAT(CAST(plan_list_price as string), CAST(payment_plan_days as string), CAST(payment_method_id as string)) as plan_sort,
            is_cancel,
            membership_expire_da